In [1]:
!pip install requests

In [1]:
import requests

In [5]:

# Set the base URL for your API
BASE_URL = 'http://127.0.0.1:8001/uploads/file_uploads'

# Define the GUID and file information
guid = 'test.zip'  # Replace with an actual GUID from your database

# Headers for the requests
headers = {
    'Content-Type': 'application/json',
}


In [6]:

# Function to update the priority
def update_priority(guid, priority):    
    url = f'{BASE_URL}/update_priority/'
    data = {
        'guid': guid,
        'priority': priority,
    }
    response = requests.post(url, json=data, headers=headers)
    print('Update Priority Response:', response.json())


In [7]:

# Function to update the status
def update_status(guid, status):
    url = f'{BASE_URL}/update_status/'
    data = {
        'guid': guid,
        'status': status,
    }
    response = requests.post(url, json=data, headers=headers)
    print('Update Status Response:', response.json())


In [ ]:

# Test updating the priority
print("Testing Update Priority API")
update_priority(guid, 1)  # Set priority to 1


In [8]:

# Test pausing the upload
print("\nTesting Pause Upload API")
update_status(guid, 'paused')



Testing Pause Upload API
Update Status Response: {'status': 'status updated'}


In [ ]:

# Test resuming the upload
print("\nTesting Resume Upload API")
update_status(guid, 'resume')



In [ ]:

# # Test canceling the upload
# print("\nTesting Cancel Upload API")
# update_status(guid, 'cancel')